<a href="https://colab.research.google.com/github/Matrixmax/Dive_into_DeepLearning/blob/main/0x19_%E8%AF%BB%E5%8F%96%E5%92%8C%E5%AD%98%E5%82%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

到目前为止，我们介绍了如何处理数据以及如何构建、训练和测试深度学习模型。然而在实际中，我们有时需要把训练好的模型部署到很多不同的设备。在这种情况下，我们可以把内存中训练好的模型参数存储在硬盘上供后续读取使用。

## 1. 读写tensor

我们可以直接使用save函数和load函数分别存储和读取Tensor。save使用Python的pickle实用程序将对象进行序列化，然后将序列化的对象保存到disk，使用save可以保存各种对象,包括模型、张量和字典等。而load使用pickle unpickle工具将pickle的对象文件反序列化为内存。

下面的例子创建了Tensor变量x，并将其存在文件名同为x.pt的文件里

In [2]:
import torch
from torch import nn

x = torch.ones(3)
torch.save(x,'x.pt')

In [3]:
# 然后我们将数据从存储的文件读回内存
x2 = torch.load('x.pt')
x2

tensor([1., 1., 1.])

In [4]:
# 存储一个tensor列表并且读回内存
y = torch.zeros(4)
torch.save([x,y],'xy.pt')
xy_list = torch.load('xy.pt')
xy_list


[tensor([1., 1., 1.]), tensor([0., 0., 0., 0.])]

In [6]:
# 存储并且读取一个从字符串映射到tensor的字典
torch.save({'x':x,'y':y},'xy_dict.pt')
xy = torch.load('xy_dict.pt')
xy

{'x': tensor([1., 1., 1.]), 'y': tensor([0., 0., 0., 0.])}

## 2.读写模型
在PyTorch中，Module的可学习参数(即权重和偏差)，模块模型包含在参数中(通过model.parameters()访问)。state_dict是一个从参数名称映射到参数Tesnor的字典对象。

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.hidden = nn.Linear(3,2)
        self.act = nn.ReLU()
        self.output = nn.Linear(2,1)
    
    def forward(self,x):
        a = self.act(self.hidden(x))
        return self.output(a)

net = MLP()
net.state_dict()

OrderedDict([('hidden.weight', tensor([[ 0.0209, -0.3549,  0.4716],
                      [ 0.5001,  0.4127, -0.1253]])),
             ('hidden.bias', tensor([-0.1757, -0.3783])),
             ('output.weight', tensor([[0.5344, 0.2990]])),
             ('output.bias', tensor([-0.6066]))])

注意，只有具有可学习参数的层(卷积层、线性层等)才有state_dict中的条目。优化器(optim)也有一个state_dict，其中包含关于优化器状态以及所使用的超参数的信息。

In [8]:
optimizer = torch.optim.SGD(net.parameters(),lr = 0.001,momentum=0.9)
optimizer.state_dict()

{'param_groups': [{'dampening': 0,
   'lr': 0.001,
   'momentum': 0.9,
   'nesterov': False,
   'params': [0, 1, 2, 3],
   'weight_decay': 0}],
 'state': {}}

### 2.2 保存和加载模型
PyTorch中保存和加载训练模型有两种常见的方法:

1. 仅保存和加载模型参数(state_dict)；
2. 保存和加载整个模型。


#### **1. 保存和加载state_dict(推荐方式)**


In [ ]:
# 保存
torch.save(net.state_dict(),PATH) #推荐的文件后缀名是pt或者pth

# 加载
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))

In [19]:
# 保存和加载实验
X = torch.randn(2,3)
Y = net(X)

path = './net.pth'
torch.save(net.state_dict(),path)

net2 = MLP()
net2.load_state_dict(torch.load(path))
Y2 = net2(X)
Y2 == Y

tensor([[True],
        [True]])

#### **2.保存和加载整个模型**

In [ ]:
# 保存
torch.save(model.PATH)

# 加载
model = torch.load(PATH)